In [1]:
import numpy as np
import itertools
import random

In [2]:
# a stores average arrival rates
a_avg = np.array([[0.2, 0.3, 0.2],
                  [0.3, 0.1, 0.25],
                  [0.2, 0.2, 0.3]])

# s stores current service
s = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

def I(n):
    A = []
    for i in range(n):
        A.append([1 if j == i else 0 for j in range(n)])
    return np.array(A)

allServiceMatrices = []
for m in itertools.permutations(I(3)):
    allServiceMatrices.append([m[i] for i in range(3)]) # allServiceMatrices.append([m[0], m[1], m[2]])

In [3]:
# strategy 1: during each interval [t - 1, t], choose a random possible configuration

def oneRandomServiceMatrix():
    return np.array(random.sample(allServiceMatrices, 1)[0])

def elementWiseMax(a1, a2):
    output = np.zeros(a1.shape)
    for i in range(len(a1)):
        for j in range(len(a1)):
            output[i, j] = max(a1[i, j], a2[i, j])
    return output

# simulate 1,000,000 times

# q stores current queue sizes
q = np.zeros((3, 3)) # this is q at time = 0

q_avg = np.zeros((3, 3)) # our goal is to find this (the average queue size for each queue)

for t in range(1, 1000001):
    a = np.random.poisson(a_avg) # this is the number of arrivals at the beginning of [t - 1, t]
    s = oneRandomServiceMatrix() # this is the service configuration for the interval [t - 1, t]
    q = elementWiseMax(q + a - s, np.zeros((3, 3))) # compute q at the beginning of [t, t + 1]
    q_avg = ((t - 1) * q_avg + q) / t

# output for strategy 1

print('Average Arrival Rate Matrix =')
print(a_avg)
print('\nService Rate Matrix =')
print(np.ones((3, 3)))
print('\nAverage Queue Size Matrix =')
print(q_avg)

Average Arrival Rate Matrix =
[[0.2  0.3  0.2 ]
 [0.3  0.1  0.25]
 [0.2  0.2  0.3 ]]

Service Rate Matrix =
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

Average Queue Size Matrix =
[[1.165367 7.283385 1.151399]
 [7.102164 0.308334 2.424406]
 [1.155635 1.150332 7.313425]]


In [4]:
# strategy 2 (maxWeight strategy):
# during each interval [t - 1, t], choose a configuration such that the queues being served have the maximum combined size

def computeInnerProduct(a1, a2):
    innerProduct = 0
    for i in range(a1.shape[0]):
        innerProduct += np.inner(a1[i], a2[i])
    return innerProduct

def maxWeightServiceMatrix(q):
    innerProducts = []
    for m in allServiceMatrices:
        innerProducts.append(computeInnerProduct(q, m))
    for m in allServiceMatrices:
        if computeInnerProduct(q, m) == max(innerProducts):
            return m

# simulate 1,000,000 times

# q stores current queue sizes
q = np.zeros((3, 3)) # this is q at time = 0

q_avg = np.zeros((3, 3)) # our goal is to find this (the average queue size for each queue)

for t in range(1, 1000001):
    a = np.random.poisson(a_avg) # this is the number of arrivals at the beginning of [t - 1, t]
    s = maxWeightServiceMatrix(q) # this is the service configuration for the interval [t - 1, t]
    q = elementWiseMax(q + a - s, np.zeros((3, 3))) # compute q at the beginning of [t, t + 1]
    q_avg = ((t - 1) * q_avg + q) / t

# output for strategy 2

print('Average Arrival Rate Matrix =')
print(a_avg)
print('\nService Rate Matrix =')
print(np.ones((3, 3)))
print('\nAverage Queue Size Matrix =')
print(q_avg)

Average Arrival Rate Matrix =
[[0.2  0.3  0.2 ]
 [0.3  0.1  0.25]
 [0.2  0.2  0.3 ]]

Service Rate Matrix =
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

Average Queue Size Matrix =
[[0.403896 0.564136 0.798271]
 [0.612048 0.193765 0.622029]
 [0.651858 0.407277 0.607556]]
